# Qualitative Data Download

In [1]:
import requests
from bs4 import BeautifulSoup
from yahoo_fin import news
from transformers import pipeline
import warnings
import pandas as pd
import concurrent.futures
import unicodedata
import re

#Initalize transformer pipelines
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english", device=0)
summarizer = pipeline("summarization", device=0)
warnings.filterwarnings('ignore')

tickers = []
with open('tickers.txt', 'r') as f:
    tickers = f.read()

tickers = tickers.split("\n")
tickers = [t.strip() for t in tickers]
#tickers

2024-11-18 21:05:04.235208: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731989104.251546   31790 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731989104.256552   31790 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-18 21:05:04.274084: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1731989114.524037   31790 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 67

In [2]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

sn_frames = {}

# Define a function to fetch news and create a DataFrame
def fetch_news(ticker):
    stock_news = news.get_yf_rss(ticker)
    return ticker, pd.DataFrame(stock_news)

# Run the news fetching in parallel
with ThreadPoolExecutor() as executor:
    futures = {executor.submit(fetch_news, t): t for t in tickers}
    
    for future in as_completed(futures):
        ticker, df = future.result()
        sn_frames[ticker] = df

print(len(sn_frames))

503


In [3]:
sn_frames.get("INTC")

,summary,summary_detail,id,guidislink,links,link,published,published_parsed,title,title_detail
0,"NVIDIA's entry into the Dow, the release of ne...","{'type': 'text/html', 'language': None, 'base'...",f20ede7f-ee5b-3c25-b566-0b2cbc777f6f,False,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://finance.yahoo.com/news/why-nvidia-join...,"Mon, 18 Nov 2024 20:05:00 +0000","(2024, 11, 18, 20, 5, 0, 0, 323, 0)",This Is Why NVIDIA Joined the Dow; And Why It'...,"{'type': 'text/plain', 'language': None, 'base..."
1,More reasons why Wall Street loves Nvidia.,"{'type': 'text/html', 'language': None, 'base'...",63b46d23-f24c-45bb-a718-22704b112184,False,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://finance.yahoo.com/news/why-nvidia-stoc...,"Mon, 18 Nov 2024 13:23:49 +0000","(2024, 11, 18, 13, 23, 49, 0, 323, 0)",Why Nvidia stock is ripe for another surge: In...,"{'type': 'text/plain', 'language': None, 'base..."
2,We recently published a list of 15 AI News Tha...,"{'type': 'text/html', 'language': None, 'base'...",85f4a347-3cc4-3052-9c46-4ae78259daa8,False,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://finance.yahoo.com/news/intel-corp-intc...,"Mon, 18 Nov 2024 13:01:22 +0000","(2024, 11, 18, 13, 1, 22, 0, 323, 0)",Intel Corp (INTC) Delays German Fab Constructi...,"{'type': 'text/plain', 'language': None, 'base..."
3,Intel Corporation ( NASDAQ:INTC ) shareholders...,"{'type': 'text/html', 'language': None, 'base'...",e806abba-5126-3c67-b4da-c5dcccc908fa,False,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://finance.yahoo.com/news/shareholders-in...,"Sun, 17 Nov 2024 11:00:40 +0000","(2024, 11, 17, 11, 0, 40, 6, 322, 0)",Shareholders in Intel (NASDAQ:INTC) have lost ...,"{'type': 'text/plain', 'language': None, 'base..."
4,We recently published a list of 15 AI News Upd...,"{'type': 'text/html', 'language': None, 'base'...",3a130134-ebb6-36a8-b4e6-3c1ccc9bea14,False,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://finance.yahoo.com/news/intel-corporati...,"Sun, 17 Nov 2024 10:30:30 +0000","(2024, 11, 17, 10, 30, 30, 6, 322, 0)",Is Intel Corporation (INTC) AI News Update tak...,"{'type': 'text/plain', 'language': None, 'base..."
5,The sellers have not only overshot their targe...,"{'type': 'text/html', 'language': None, 'base'...",8bbc4dc3-a334-3dbb-a84c-8ca7a1ce2982,False,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://www.fool.com/investing/2024/11/17/3-gr...,"Sun, 17 Nov 2024 10:15:00 +0000","(2024, 11, 17, 10, 15, 0, 6, 322, 0)","3 Growth Stocks Down 84%, 28%, and 97% to Buy ...","{'type': 'text/plain', 'language': None, 'base..."
6,Intel (NASDAQ: INTC) was once considered a sta...,"{'type': 'text/html', 'language': None, 'base'...",6ba0429a-88b5-3789-b359-6d24fe5d0e9b,False,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://www.fool.com/investing/2024/11/17/coul...,"Sun, 17 Nov 2024 10:07:00 +0000","(2024, 11, 17, 10, 7, 0, 6, 322, 0)",Could Intel Stock Help You Become a Millionaire?,"{'type': 'text/plain', 'language': None, 'base..."
7,We recently compiled a list of the 15 Trending...,"{'type': 'text/html', 'language': None, 'base'...",c46bb1a4-ca75-38dc-8280-4466db47e9a9,False,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://finance.yahoo.com/news/advanced-micro-...,"Sat, 16 Nov 2024 06:42:07 +0000","(2024, 11, 16, 6, 42, 7, 5, 321, 0)","Advanced Micro Devices, Inc. (AMD) Unveils Ryz...","{'type': 'text/plain', 'language': None, 'base..."
8,We recently made a list of UBS’ Bottom Quant S...,"{'type': 'text/html', 'language': None, 'base'...",33a0bb88-2678-3378-83f6-822b15142680,False,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://finance.yahoo.com/news/ubs-intel-corpo...,"Fri, 15 Nov 2024 23:51:35 +0000","(2024, 11, 15, 23, 51, 35, 4, 320, 0)",UBS: Intel Corporation (NASDAQ:INTC) Is A Bott...,"{'type': 'text/plain', 'language': None, 'base..."
9,Nvidia will report its Q3 earnings after the b...,"{'type': 'text/html', 'language': None

# Cleaning up the Data

In [ ]:
def basic_cleanup(text):
    '''Perform basic cleanup on article text'''
    cleaned_text = unicodedata.normalize('NFKD', text).encode('ascii','ignore').decode('utf-8', 'ignore')
    cleaned_text = re.sub(r'[^a-zA-Z0-9.,;:!?\'\"()\s]', '', cleaned_text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text

def summarize_article(article_text):
    summary = summarizer(article_text[:2056], max_length=100, min_length=30, do_sample=False)
    return summary[0]['summary_text']

def add_sentiment(df:pd.DataFrame):
    setiments = df['article_text'].apply(lambda text: sentiment_pipeline(text[:2056])[0]['label'] if text != "N/A" else "UNKNOWN")
    df['sentiment'] = setiments
    return df

def get_article_text(url):
    try:     
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"
        }
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            paragprahs = soup.find_all('p')
            article_text = ' '.join([p.get_text() for p in paragprahs])
            article_text = basic_cleanup(article_text)
            return article_text
        else:
            print(f"Failed to get article text from {url} got a response code of {response.status_code}")
            return "N/A"
    except Exception as e:
        print(f"Failed to get article text from {url}: {e}")
        return "N/A"

def preprocess_data(df:pd.DataFrame) -> pd.DataFrame:
    try:
        df = df[['summary', 'link', 'published', 'title']]
        with concurrent.futures.ThreadPoolExecutor() as executor:
            article_texts = list(executor.map(get_article_text, df['link']))
        df['article_text'] = article_texts
        df = add_sentiment(df)
        return df
    except Exception as e:
        return f"Could not process data {e}"

In [5]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
from random import uniform

# Define a function to preprocess and handle the result
def preprocess_and_update(ticker, data):
    df = preprocess_data(data)
    if isinstance(df, str):
        print(df)
    return ticker, df if not isinstance(df, str) else None

def preprocess_and_update_with_retry(ticker, data, max_retries=5):
    delay = 1
    for i in range(max_retries):
        try:
            # Add a random delay to reduce server load
            time.sleep(uniform(1, 3))  # 1-3 seconds random delay
            return ticker, preprocess_and_update(ticker, data)
        except Exception as e:
            print(f"Error for {ticker}: {e}. Retrying in {delay} seconds...")
            time.sleep(delay)
            delay *= 2  # Exponential backoff
    print(f"Failed to fetch data for {ticker} after {max_retries} retries.")
    return ticker, None

# Run preprocessing in parallel
with ThreadPoolExecutor(max_workers=4) as executor:
    futures = {
        executor.submit(preprocess_and_update_with_retry, ticker, data): ticker 
        for ticker, data in sn_frames.items()
    }
    
    for future in as_completed(futures):
        ticker, result = future.result()
        sn_frames[ticker] = result


Failed to get article text from https://www.wsj.com/health/pharma/what-the-nfl-draft-can-teach-investors-about-big-pharma-d61268f9?.tsrc=rss got a response code of 401
Failed to get article text from https://www.marketwatch.com/story/these-are-the-s-p-500s-biggest-laggards-for-october-down-by-as-much-as-29-bd417f5d?.tsrc=rss got a response code of 401
Failed to get article text from https://finance.yahoo.com/m/8e993564-4856-3bca-8091-dc1a7a706901/amgen-drops-7%25-after-a-%27big.html?.tsrc=rss got a response code of 404
Failed to get article text from https://finance.yahoo.com/m/51bd9a38-c381-341a-a31d-175d7f8716cc/s%26p-500-giant-leads-5-stocks.html?.tsrc=rss got a response code of 404
Failed to get article text from https://finance.yahoo.com/news/ansys-inc-nasdaq-anss-financials-191929469.html?.tsrc=rss got a response code of 404
Failed to get article text from https://finance.yahoo.com/news/aon-plc-price-target-raised-122537662.html?.tsrc=rss got a response code of 404
Failed to get 

In [24]:
sn_frames.get("INTC")[1]

,summary,link,published,title,article_text
0,"NVIDIA's entry into the Dow, the release of ne...",https://finance.yahoo.com/news/why-nvidia-join...,"Mon, 18 Nov 2024 20:05:00 +0000",This Is Why NVIDIA Joined the Dow; And Why It'...,We are experiencing some temporary issues. The...
1,More reasons why Wall Street loves Nvidia.,https://finance.yahoo.com/news/why-nvidia-stoc...,"Mon, 18 Nov 2024 13:23:49 +0000",Why Nvidia stock is ripe for another surge: In...,We are experiencing some temporary issues. The...
2,We recently published a list of 15 AI News Tha...,https://finance.yahoo.com/news/intel-corp-intc...,"Mon, 18 Nov 2024 13:01:22 +0000",Intel Corp (INTC) Delays German Fab Constructi...,We are experiencing some temporary issues. The...
3,Intel Corporation ( NASDAQ:INTC ) shareholders...,https://finance.yahoo.com/news/shareholders-in...,"Sun, 17 Nov 2024 11:00:40 +0000",Shareholders in Intel (NASDAQ:INTC) have lost ...,We are experiencing some temporary issues. The...
4,We recently published a list of 15 AI News Upd...,https://finance.yahoo.com/news/intel-corporati...,"Sun, 17 Nov 2024 10:30:30 +0000",Is Intel Corporation (INTC) AI News Update tak...,We are experiencing some temporary issues. The...
5,The sellers have not only overshot their targe...,https://www.fool.com/investing/2024/11/17/3-gr...,"Sun, 17 Nov 2024 10:15:00 +0000","3 Growth Stocks Down 84%, 28%, and 97% to Buy ...","Founded in 1993, The Motley Fool is a financia..."
6,Intel (NASDAQ: INTC) was once considered a sta...,https://www.fool.com/investing/2024/11/17/coul...,"Sun, 17 Nov 2024 10:07:00 +0000",Could Intel Stock Help You Become a Millionaire?,"Founded in 1993, The Motley Fool is a financia..."
7,We recently compiled a list of the 15 Trending...,https://finance.yahoo.com/news/advanced-micro-...,"Sat, 16 Nov 2024 06:42:07 +0000","Advanced Micro Devices, Inc. (AMD) Unveils Ryz...",We are experiencing some temporary issues. The...
8,We recently made a list of UBS’ Bottom Quant S...,https://finance.yahoo.com/news/ubs-intel-corpo...,"Fri, 15 Nov 2024 23:51:35 +0000",UBS: Intel Corporation (NASDAQ:INTC) Is A Bott...,We are experiencing some temporary issues. The...
9,Nvidia will report its Q3 earnings after the b...,https://finance.yahoo.com/news/nvidia-to-repor...,"Fri, 15 Nov 2024 18:58:00 +0000",Nvidia to report Q3 earnings Wednesday as AI f...,We are experiencing some temporary issues. The...


In [25]:
filler_texts = [
    "We are experiencing some temporary issues." ,
    "The market data on this page is currently delayed.",
    "Please bear with us as we address this and restore your personalized lists.",
    "Founded in 1993, The Motley Fool is a financial services company dedicated to making the world smarter, happier, and richer.",
    "The Motley Fool reaches millions of people every month through our premium investing solutions, free guidance and market analysis on Fool.com, toprated podcasts, and nonprofit The Motley Fool Foundation.",
    "Founded in 1993, The Motley Fool is a financial services company dedicated to making the world smarter, happier, and richer. The Motley Fool reaches millions of people every month through our premium investing solutions, free guidance and market analysis on Fool.com, toprated podcasts, and nonprofit The Motley Fool Foundation.",
    "You're reading a free article with opinions that may differ from The Motley Fool's Premium Investing Services.",
    "Become a Motley Fool member today to get instant access to our top analyst recommendations, indepth research, investing resources, and more.",
    "Learn More Key Points",
    "Read the latest financial and business news from Yahoo Finance Sign in to access your portfolio",
    "The Motley Fool has a disclosure policy.",
    "Related Articles Invest better with The Motley Fool.",
    "Get stock recommendations, portfolio guidance, and more from The Motley Fool's premium services.",
    "Making the world smarter, happier, and richer.",
    "1995 2024 The Motley Fool. All rights reserved. Market data powered by Xignite and Polygon.io.",
    "About The Motley Fool Our Services Around the Globe Free Tools Affiliates Friends",
    "Free Stock Analysis Report To read this article on Zacks.com click here.",
    "Zacks Investment Research Sign in to access your portfolio",
    "To watch more expert insights and analysis on the latest market action, check out more Catalysts here.",
    "This post was written", 
    "Sign in to access your portfolio"]

In [26]:
import re

# Create a single regular expression pattern that matches any of the filler texts
filler_pattern = '|'.join(map(re.escape, filler_texts))

# Pattern to match unnecessary whitespace, tabs, and escape characters
whitespace_pattern = r'\s+'

# Process each frame in parallel, replacing filler texts and stripping whitespace
for ticker, frame in sn_frames.items():
    try:
        # Replace filler text and strip extra whitespace in all articles for the given ticker
        frame[1]['article_text'] = frame[1]['article_text'].apply(
            lambda text: re.sub(whitespace_pattern, ' ', re.sub(filler_pattern, '', text)).strip() if isinstance(text, str) else text
        )
    except Exception as e:
        continue

In [27]:
sn_frames.get("INTC")[1]['article_text']

0     NVIDIA Corporations NVDA graphic processing un...
1     Listen and subscribe to Opening Bid on Apple P...
2     We recently published a list of 15 AI News Tha...
3     Intel Corporation (NASDAQ:INTC) shareholders s...
4     We recently published a list of 15 AI News Upd...
5     The sellers have not only overshot their targe...
6     Intel (INTC 2.01) was once considered a stable...
7     We recently compiled a list of the 15 Trending...
8     We recently made a list of UBS Bottom Quant St...
9     Nvidia (NVDA) will report its Q3 earnings afte...
10    We recently compiled a list of the 15 AI News ...
11    Intel (INTC) is one of the stocks most watched...
12    (Bloomberg) The Biden administration finalized...
13    Advanced Micro Devices (AMD), Intel (INTC), an...
14    Intel (NASDAQ:INTC) used to be the undisputed ...
15    SANTA CLARA, Calif., November 13, 2024(BUSINES...
16    Intel more than halved its stake in Astera Lab...
17    We recently compiled a list of the 14 AI N

In [33]:
def process_ticker(ticker, frame):
    try:
        df = frame[1]
        if df is not None:
            df = add_sentiment(df)
        return ticker, df
    except Exception as e:
        return ticker, None

# Parallelize the loop
with ThreadPoolExecutor() as executor:
    futures = {
        executor.submit(process_ticker, ticker, frame): ticker
        for ticker, frame in sn_frames.items()
    }
    
    for future in as_completed(futures):
        ticker, df = future.result()
        sn_frames[ticker] = df

In [45]:
for ticker, frame in sn_frames.items():
    if frame is None:
        print(ticker)

A
ABBV
AOS
ALGN
ACN
APD
ABNB
MMM
AES
AMD
ALB
AKAM
ADBE
ARE
AFL
ABT
ALL
LNT
GOOG
ALLE
MO
AEP
AMTM
AMCR
AXP
AMZN
AWK
AIG
AMT
AMP
AEE
AME
ADI
APH
AMGN
ANSS
AON
APA
AAPL
AMAT
APTV
ADM
ANET
ACGL
AJG
AIZ
T
ATO
AZO
AVB
ADP
ADSK
AVY
AXON
BKR
BALL
BAC
BRK.
BDX
BAX
BBY
TECH
BIIB
BLK
BX
BA
BK
BKNG
BWA
BSX
AVGO
BMY
BR
BF.B
BRO
BLDR
CHRW
BG
BXP
CDNS
CZR
CPT
COF
CPB
CAH
KMX
CARR
CCL
CTLT
CDW
CBRE
CAT
CBOE
COR
CE
CNC
CF
CNP
SCHW
CRL
CHTR
CVX
CMG
CB
CHD
CTAS
CINF
CI
CSCO
C
CFG
CLX
CME
CMCS

